<a href="https://colab.research.google.com/github/Anish32/Recommented-system/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df=pd.read_csv('/content/fashion_products.csv')
df.head()

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M


In [ ]:
df.describe()

,User ID,Product ID,Price,Rating
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,50.41900,500.500000,55.785000,2.993135
std,28.78131,288.819436,26.291748,1.153185
min,1.00000,1.000000,10.000000,1.000967
25%,26.00000,250.750000,33.000000,1.992786
50%,50.00000,500.500000,57.000000,2.984003
75%,75.00000,750.250000,78.250000,3.985084
max,100.00000,1000.000000,100.000000,4.987964


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   User ID       1000 non-null   int64  
 1   Product ID    1000 non-null   int64  
 2   Product Name  1000 non-null   object 
 3   Brand         1000 non-null   object 
 4   Category      1000 non-null   object 
 5   Price         1000 non-null   int64  
 6   Rating        1000 non-null   float64
 7   Color         1000 non-null   object 
 8   Size          1000 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 70.4+ KB


In [ ]:
df.shape

(1000, 9)

In [ ]:
df.isnull().sum()

,0
User ID,0
Product ID,0
Product Name,0
Brand,0
Category,0
Price,0
Rating,0
Color,0
Size,0


In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(df[['User ID', 'Product ID', 'Rating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
model = SVD()
model.fit(trainset)

In [ ]:
predictions = model.test(testset)
print("\nRoot Mean Square Error (RMSE):", rmse(predictions))

RMSE: 1.1255

Root Mean Square Error (RMSE): 1.1255309706083663


In [ ]:
predicted_ratings = [pred.est for pred in predictions]

In [ ]:
def recommend_products(user_id, num_recommendations=5):
    unique_products = df['Product ID'].unique()
    rated_products = df[df['User ID'] == user_id]['Product ID'].values
    products_to_predict = [p for p in unique_products if p not in rated_products]

    predictions = [model.predict(user_id, p) for p in products_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)

    top_products = [pred.iid for pred in predictions[:num_recommendations]]
    return df[df['Product ID'].isin(top_products)][['Product Name', 'Brand', 'Category', 'Price']]


In [ ]:
user_id = 20
recommendations = recommend_products(user_id)
recommendations

,Product Name,Brand,Category,Price
125,T-shirt,Nike,Kids' Fashion,55
144,T-shirt,Zara,Kids' Fashion,60
211,T-shirt,Adidas,Women's Fashion,71
299,Shoes,Adidas,Kids' Fashion,57
956,Jeans,Nike,Men's Fashion,57


In [ ]:
from surprise.accuracy import rmse


predictions = model.test(testset)

print("\nRoot Mean Square Error (RMSE):", rmse(predictions))

RMSE: 1.1274

Root Mean Square Error (RMSE): 1.127429610161508


In [ ]:
actual_ratings = [pred.r_ui for pred in predictions]
predicted_ratings = [pred.est for pred in predictions]

In [ ]:
from sklearn.metrics import r2_score
r_squared = r2_score(actual_ratings, predicted_ratings)
print("R-squared:", r_squared)

R-squared: -0.02417298715971139
